In [1]:
import tensorflow as tf

In [2]:
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.datasets import imdb

In [3]:
from google.colab import files
uploaded = files.upload()

Saving stocktwits_cleaned_messages_nlp.csv to stocktwits_cleaned_messages_nlp.csv


In [41]:
import pandas as pd
data = pd.read_csv("stocktwits_cleaned_messages_nlp.csv",encoding='ISO-8859-1')

In [42]:
data.head(10)

,message,sentiment
0,tsla funny people saying going crash pullback ...,Bullish
1,tsla anyone else feel pinned last week,Bullish
2,tsla anytime watchout,Bearish
3,tsla bears scared letting bring little extra m...,Bullish
4,tsla day reckoning bulls,Bullish
5,tsla kicked worry battery day foot,Bearish
6,tsla cnbc shut fu cretins need crawl back rocks,Bullish
7,tsla crash coming,Bearish
8,tsla two ol bears chat,Bullish
9,tsla see bear trying get lose faith read actua...,Bullish


In [43]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [44]:
data.rename(columns={"message": "Text", "sentiment": "Label"},inplace=True)

In [33]:
data.head()

,Text,Label
0,tsla funny people saying going crash pullback ...,Bullish
1,tsla anyone else feel pinned last week,Bullish
2,tsla anytime watchout,Bearish
3,tsla bears scared letting bring little extra m...,Bullish
4,tsla day reckoning bulls,Bullish


In [45]:
for l in data.Label:
    if l == "Bullish":
        data.Label.replace("Bullish",1,inplace=True)
    elif l == "Bearish":
        data.Label.replace("Bearish",0,inplace=True)
       

In [46]:
data.head()

,Text,Label
0,tsla funny people saying going crash pullback ...,1
1,tsla anyone else feel pinned last week,1
2,tsla anytime watchout,0
3,tsla bears scared letting bring little extra m...,1
4,tsla day reckoning bulls,1


In [ ]:
df=data['sentiment']
df


0       Bullish
1       Bullish
2       Bearish
3       Bullish
4       Bullish
         ...   
2160    Bullish
2161    Bullish
2162    Bullish
2163    Bullish
2164    Bullish
Name: sentiment, Length: 2165, dtype: object

In [47]:
pos = []
neg = []
for l in data.Label:
    if l == 0:
        pos.append(0)
        neg.append(1)
    elif l == 1:
        pos.append(1)
        neg.append(0)

In [50]:
data['Pos']= pos
data['Neg']= neg
data.head()

,Text,Label,Pos,Neg
0,tsla funny people saying going crash pullback ...,1,1,0
1,tsla anyone else feel pinned last week,1,1,0
2,tsla anytime watchout,0,0,1
3,tsla bears scared letting bring little extra m...,1,1,0
4,tsla day reckoning bulls,1,1,0


In [52]:
from nltk import word_tokenize, WordNetLemmatizer
tokens = [word_tokenize(sen) for sen in data.Text] 

In [55]:
data['tokens']=tokens

In [56]:
data = data[['Text', 'tokens', 'Label', 'Pos', 'Neg']]

In [57]:
data.head()

,Text,tokens,Label,Pos,Neg
0,tsla funny people saying going crash pullback ...,"[tsla, funny, people, saying, going, crash, pu...",1,1,0
1,tsla anyone else feel pinned last week,"[tsla, anyone, else, feel, pinned, last, week]",1,1,0
2,tsla anytime watchout,"[tsla, anytime, watchout]",0,0,1
3,tsla bears scared letting bring little extra m...,"[tsla, bears, scared, letting, bring, little, ...",1,1,0
4,tsla day reckoning bulls,"[tsla, day, reckoning, bulls]",1,1,0


In [59]:
from sklearn.model_selection import train_test_split

In [60]:
data_train, data_test = train_test_split(data, test_size=0.10, random_state=42)

In [61]:
all_training_words = [word for tokens in data_train["tokens"] for word in tokens]
training_sentence_lengths = [len(tokens) for tokens in data_train["tokens"]]
TRAINING_VOCAB = sorted(list(set(all_training_words)))
print("%s words total, with a vocabulary size of %s" % (len(all_training_words), len(TRAINING_VOCAB)))
print("Max sentence length is %s" % max(training_sentence_lengths))

23113 words total, with a vocabulary size of 4664
Max sentence length is 118


In [62]:
all_test_words = [word for tokens in data_test["tokens"] for word in tokens]
test_sentence_lengths = [len(tokens) for tokens in data_test["tokens"]]
TEST_VOCAB = sorted(list(set(all_test_words)))
print("%s words total, with a vocabulary size of %s" % (len(all_test_words), len(TEST_VOCAB)))
print("Max sentence length is %s" % max(test_sentence_lengths))

2541 words total, with a vocabulary size of 1167
Max sentence length is 73


In [68]:
!wget -P /root/input/ -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

--2020-11-06 15:32:46--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.49.102
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.49.102|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1647046227 (1.5G) [application/x-gzip]
Saving to: ‘/root/input/GoogleNews-vectors-negative300.bin.gz’

GoogleNews-vectors- 100%[===================>]   1.53G  62.8MB/s    in 24s     

2020-11-06 15:33:11 (64.6 MB/s) - ‘/root/input/GoogleNews-vectors-negative300.bin.gz’ saved [1647046227/1647046227]



In [71]:
from gensim.models import KeyedVectors

In [72]:
EMBEDDING_FILE = '/root/input/GoogleNews-vectors-negative300.bin.gz' # from above
word2vec = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)

In [64]:
from __future__ import division, print_function
from gensim import models
from keras.callbacks import ModelCheckpoint
from keras.layers import Dense, Dropout, Reshape, Flatten, concatenate, Input, Conv1D, GlobalMaxPooling1D, Embedding
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import os
import collections
import re
import string

In [74]:
word2vec_path = '/root/input/GoogleNews-vectors-negative300.bin.gz'
word2vec = models.KeyedVectors.load_word2vec_format(word2vec_path, binary=True)

In [75]:
def get_average_word2vec(tokens_list, vector, generate_missing=False, k=300):
    if len(tokens_list)<1:
        return np.zeros(k)
    if generate_missing:
        vectorized = [vector[word] if word in vector else np.random.rand(k) for word in tokens_list]
    else:
        vectorized = [vector[word] if word in vector else np.zeros(k) for word in tokens_list]
    length = len(vectorized)
    summed = np.sum(vectorized, axis=0)
    averaged = np.divide(summed, length)
    return averaged

def get_word2vec_embeddings(vectors, clean_comments, generate_missing=False):
    embeddings = clean_comments['tokens'].apply(lambda x: get_average_word2vec(x, vectors, 
                                                                                generate_missing=generate_missing))
    return list(embeddings)

In [76]:
training_embeddings = get_word2vec_embeddings(word2vec, data_train, generate_missing=True)

In [77]:
MAX_SEQUENCE_LENGTH = 50
EMBEDDING_DIM = 300

In [81]:
tokenizer = Tokenizer(num_words=len(TRAINING_VOCAB), lower=True, char_level=False)
tokenizer.fit_on_texts(data_train["Text"].tolist())
training_sequences = tokenizer.texts_to_sequences(data_train["Text"].tolist())

train_word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(train_word_index))

Found 4664 unique tokens.


In [82]:
train_cnn_data = pad_sequences(training_sequences, maxlen=MAX_SEQUENCE_LENGTH)

In [83]:
train_embedding_weights = np.zeros((len(train_word_index)+1, EMBEDDING_DIM))
for word,index in train_word_index.items():
    train_embedding_weights[index,:] = word2vec[word] if word in word2vec else np.random.rand(EMBEDDING_DIM)
print(train_embedding_weights.shape)

(4665, 300)


In [85]:
test_sequences = tokenizer.texts_to_sequences(data_test["Text"].tolist())
test_cnn_data = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)

In [86]:
def ConvNet(embeddings, max_sequence_length, num_words, embedding_dim, labels_index):
    
    embedding_layer = Embedding(num_words,
                            embedding_dim,
                            weights=[embeddings],
                            input_length=max_sequence_length,
                            trainable=False)
    
    sequence_input = Input(shape=(max_sequence_length,), dtype='int32')
    embedded_sequences = embedding_layer(sequence_input)

    convs = []
    filter_sizes = [2,3,4,5,6]

    for filter_size in filter_sizes:
        l_conv = Conv1D(filters=200, kernel_size=filter_size, activation='relu')(embedded_sequences)
        l_pool = GlobalMaxPooling1D()(l_conv)
        convs.append(l_pool)


    l_merge = concatenate(convs, axis=1)

    x = Dropout(0.1)(l_merge)  
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.2)(x)
    preds = Dense(labels_index, activation='sigmoid')(x)

    model = Model(sequence_input, preds)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['acc'])
    model.summary()
    return model

In [88]:
label_names = ['Pos', 'Neg']

In [89]:
y_train = data_train[label_names].values

In [90]:
x_train = train_cnn_data
y_tr = y_train

In [91]:
model = ConvNet(train_embedding_weights, MAX_SEQUENCE_LENGTH, len(train_word_index)+1, EMBEDDING_DIM, 
                len(list(label_names)))

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 50)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 50, 300)      1399500     input_1[0][0]                    
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 49, 200)      120200      embedding[0][0]                  
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 48, 200)      180200      embedding[0][0]                  
_______________________________________________________________________________________

In [100]:
num_epochs = 10
batch_size = 34

In [101]:
hist = model.fit(x_train, y_tr, epochs=num_epochs, validation_split=0.1, shuffle=True, batch_size=batch_size)

Epoch 1/10
52/52 [==============================] - 9s 169ms/step - loss: 0.1764 - acc: 0.9258 - val_loss: 0.4978 - val_acc: 0.7949
Epoch 2/10
52/52 [==============================] - 9s 168ms/step - loss: 0.1161 - acc: 0.9549 - val_loss: 0.5350 - val_acc: 0.8000
Epoch 3/10
52/52 [==============================] - 9s 170ms/step - loss: 0.0910 - acc: 0.9658 - val_loss: 0.5739 - val_acc: 0.8154
Epoch 4/10
52/52 [==============================] - 9s 169ms/step - loss: 0.0986 - acc: 0.9606 - val_loss: 0.8226 - val_acc: 0.7949
Epoch 5/10
52/52 [==============================] - 9s 168ms/step - loss: 0.0763 - acc: 0.9703 - val_loss: 0.7263 - val_acc: 0.8000
Epoch 6/10
52/52 [==============================] - 9s 169ms/step - loss: 0.0573 - acc: 0.9778 - val_loss: 0.6765 - val_acc: 0.8205
Epoch 7/10
52/52 [==============================] - 9s 168ms/step - loss: 0.0549 - acc: 0.9766 - val_loss: 0.7175 - val_acc: 0.7846
Epoch 8/10
52/52 [==============================] - 9s 169ms/step - loss: 0.

In [102]:
hist = model.fit(x_train, y_tr, epochs=num_epochs, validation_split=0.1, shuffle=True, batch_size=batch_size)

Epoch 1/10
52/52 [==============================] - 9s 170ms/step - loss: 0.0453 - acc: 0.9817 - val_loss: 1.0342 - val_acc: 0.8000
Epoch 2/10
52/52 [==============================] - 9s 169ms/step - loss: 0.0450 - acc: 0.9772 - val_loss: 1.0126 - val_acc: 0.7846
Epoch 3/10
52/52 [==============================] - 9s 169ms/step - loss: 0.0397 - acc: 0.9840 - val_loss: 0.9701 - val_acc: 0.7949
Epoch 4/10
52/52 [==============================] - 9s 169ms/step - loss: 0.0358 - acc: 0.9863 - val_loss: 1.1693 - val_acc: 0.8154
Epoch 5/10
52/52 [==============================] - 9s 169ms/step - loss: 0.0291 - acc: 0.9897 - val_loss: 1.0199 - val_acc: 0.7590
Epoch 6/10
52/52 [==============================] - 9s 168ms/step - loss: 0.0415 - acc: 0.9852 - val_loss: 1.0984 - val_acc: 0.8103
Epoch 7/10
52/52 [==============================] - 9s 168ms/step - loss: 0.0343 - acc: 0.9875 - val_loss: 1.6050 - val_acc: 0.8256
Epoch 8/10
52/52 [==============================] - 9s 169ms/step - loss: 0.

In [103]:
predictions = model.predict(test_cnn_data, batch_size=1024, verbose=1)

1/1 [==============================] - 0s 1ms/step


In [104]:
labels = [1, 0]

In [105]:
prediction_labels=[]
for p in predictions:
    prediction_labels.append(labels[np.argmax(p)])

In [106]:
sum(data_test.Label==prediction_labels)/len(prediction_labels)

0.8202764976958525

In [107]:
data_test.Label.value_counts()

1    165
0     52
Name: Label, dtype: int64